# NoteBook train model

## Import thư viện

In [1]:
import pickle

import numpy as np

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import Dense, MaxPool2D, Conv2D, BatchNormalization, Dropout, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam


## Preprocess data

Lấy data từ file thư mục chứa 

In [2]:
with open('data_processed\data.pkl', 'rb') as f:
    data = pickle.load(f)
with open('data_processed\data.pkl', 'rb') as f:
    lable = pickle.load(f)

In [3]:
# Encoder lable
encoder = OneHotEncoder(sparse_output= False)
lable_processed = encoder.fit_transform(lable)
# Chuẩn hóa data đầu vào
data_processed = data.astype('float32') / 255
data_processed = np.expand_dims(data_processed, axis= -1) # Mở rộng đầu vào vì yêu cầu phải là 3D mà do shape là (128, 128) ->> (128, 128, 1)

ValueError: Found array with dim 3. None expected <= 2.

In [7]:
data_processed.shape

(500, 128, 128, 1)

Lưu categories lại vào file để tiện dùng lại để gán lable

In [8]:
with open('categories.pkl', 'wb') as f:
    pickle.dump(encoder.categories_, f)

In [9]:
with open('categories.pkl', 'rb') as f:
    cat = pickle.load(f)

lb = np.array(cat[0])
num_class = lb.size # Số class dùng để phân biệt

Chia tập train test

In [10]:

xtrain, xtest, ytrain, ytest = train_test_split(data_processed, lable_processed, test_size= 0.2)

## Tạo model và train

In [ ]:
model_cnn = Sequential([
    Conv2D(8, (3, 3), activation='relu', input_shape=(128, 128, 1)), # Yêu cầu input_shape phải là 3D
    MaxPool2D((2, 2)),
    
    Conv2D(16, (3, 3), activation='relu'),
    MaxPool2D((2, 2)),
    
    Flatten(),
    
    Dense(128, activation='relu'),
    Dropout(0.5),
    
    Dense(num_class, activation= 'softmax'),
])
model_cnn.compile(optimizer= Adam(learning_rate= 0.001), loss= 'categorical_crossentropy', metrics= ['acc', 'precision', 'recall'])

model_cnn.summary() 

c:\Users\levie\.conda\envs\py310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 8)    │            80 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 8)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 16)     │         1,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 14400)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     3,686,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,688,161 (14.07 MB)

 Trainable params: 3,688,161 (14.07 MB)

 Non-trainable params: 0 (0.00 B)

Tiến hành train model

In [13]:
model_cnn.fit(xtrain, ytrain, epochs= 10, validation_data = (xtest, ytest) , batch_size= 16)

Epoch 1/10


c:\Users\levie\.conda\envs\py310\lib\site-packages\keras\src\ops\nn.py:907: UserWarning: You are using a softmax over axis -1 of a tensor of shape (16, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
c:\Users\levie\.conda\envs\py310\lib\site-packages\keras\src\losses\losses.py:33: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(16, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - acc: 1.0000 - loss: 0.0000e+00 - precision: 1.0000 - recall: 1.0000

c:\Users\levie\.conda\envs\py310\lib\site-packages\keras\src\ops\nn.py:907: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
c:\Users\levie\.conda\envs\py310\lib\site-packages\keras\src\losses\losses.py:33: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 67ms/step - acc: 1.0000 - loss: 0.0000e+00 - precision: 1.0000 - recall: 1.0000 - val_acc: 1.0000 - val_loss: 0.0000e+00 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - acc: 1.0000 - loss: 0.0000e+00 - precision: 1.0000 - recall: 1.0000 - val_acc: 1.0000 - val_loss: 0.0000e+00 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - acc: 1.0000 - loss: 0.0000e+00 - precision: 1.0000 - recall: 1.0000 - val_acc: 1.0000 - val_loss: 0.0000e+00 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - acc: 1.0000 - loss: 0.0000e+00 - precision: 1.0000 - recall: 1.0000 - val_acc: 1.0000 - val_loss: 0.0000e+00 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - acc: 1.0000 - loss: 0.0000e+00 - precision: 1.0000 - recall: 1.0000 - val_acc: 1.0000 - val_loss: 0.0000e+00 - val_precision: 1.00

## Lưu model lại

In [14]:
model_cnn.save('model_cnn.h5', include_optimizer = True) 